<a href="https://colab.research.google.com/github/Takumi173/JPMA2023TF1-1/blob/main/JPMA2023_case3_(1)_%E6%B7%BB%E4%BB%98%E6%96%87%E6%9B%B8%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E6%BA%96%E5%82%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続とパスの指定

In [1]:
# データ受け渡しのためにGoogle Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# データ保存ディレクトリの指定（ディレクトリは事前に用意する）
datadir = '/content/drive/MyDrive/JPMA2023_case3/'

# マイ医薬品集作成サービスからダウンロードした添付文書データのパスを指定（ファイルは事前に用意する。）
PackageInserts_Path = '/content/drive/MyDrive/PackageInserts/pmda_all_sgml_xml_20231026.zip'

# 薬価基準収載品目リストの4つのxlsxファイル（内容薬；注射薬；外用薬；歯科用薬剤）を格納したディレクトリを指定（ファイルは事前に用意する。）
PriceLists_Path = '/content/drive/MyDrive/PriceList'

Mounted at /content/drive


## ライブラリとパッケージのインストール

In [2]:
import glob
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

# 添付文書のデータ読み込み

## XML/SGMLのパスを取得

In [3]:
# 添付文書データファイルを展開
!unzip -O utf8 $PackageInserts_Path -d 'PackageInserts'

# XML/SGMLのパス一覧を取得
xmls = glob.glob('PackageInserts/*/*/*/*.xml', recursive=True)
sgmls = glob.glob('PackageInserts/*/*/*/*.sgm', recursive=True)

# 件数の確認
print('xmls  : ' + str(len(xmls)))
print('sgmls : ' + str(len(sgmls)))

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig05.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig06.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig07.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig08.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig09.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig10.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig11.gif  
  inflating: PackageInserts/pmda_all_sgml_xml_20231026/SGML_XML/ロスバスタチン錠５ｍｇ「ＪＧ」/580591_2189017F1057_1_11_fig12.gif  
  inflating: PackageInserts/pmda

## XMLファイルの読み込み

In [4]:
# XMLから必要なデータを取得する関数を定義
def ReadXML (FileNames):
  return_list=[]
  for Filename in tqdm(FileNames, total = len(FileNames)):

    # BeautifulSoupでパース
    with open (Filename,"r")as f:
      tmp =f.read()
      contents = tmp.replace('\u3000', ' ').replace('&enter;','\n')        # Unicode空白と改行文字を置換
      soup = BeautifulSoup(contents, 'xml')

    # 読み込むタグを指定
    TargetTags = [
        'PackageInsertNo',
        'ApprovalBrandName',
        'YJCode',
        'TrademarkInEnglish',
        'GenericName',
        'IndicationsOrEfficacy',
        'DoseAdmin',
        'GeneralName',
    ]

    row = []

    # 指定したタグの内容をリストで取得
    for Target in TargetTags:
      t =[]
      Tags = soup.find_all(Target)

      # タグが複数存在する場合はすべて取得
      for Tag in Tags:
          Texts = Tag.text.split("\n")
          for Text in Texts:
            if Text.strip() != "":
              t.append(Text.strip())

      row.append(t)

    return_list.append(row)
  return return_list

In [5]:
# XMLファイルから必要なデータを読み込んで保存
Lists_XML = ReadXML(xmls)
df=pd.DataFrame(Lists_XML)
df.to_csv("ReadXML.csv")

  0%|          | 0/10572 [00:00<?, ?it/s]

## SGMLファイルの読み込み

In [6]:
# SGMLから必要なデータを取得する関数を定義
def ReadSGML (FileNames):
  return_list=[]
  for n, Filename in tqdm(enumerate(FileNames), total = len(FileNames)):

    # BeautifulSoupでパース
    with open (Filename,"r", encoding="shift-jis")as f:
      tmp =f.read()
      contents = tmp.replace('\u3000', ' ').replace('&enter;','\n')        # Unicode空白と改行文字を置換
      soup = BeautifulSoup(contents, 'lxml')

    # 不要な要素を削除
    for tag in soup.findAll(["variablelabel","serialno"]):
        tag.decompose()

    # 読み込むタグを指定
    TargetTags = [
        'PackageInsertNo',
        'approvalbrandname',
        'yjcode',
        'TrademarkInEnglish',
        'genericname',
        'indicationsorefficacy',
        'doseadmin',
        'physchemofactingredients',
    ]

    row = []

    # 指定したタグの内容をリストで取得
    for Target in TargetTags:
      t =[]
      Tags = soup.find_all(Target)

      # タグが複数存在する場合はすべて取得
      for Tag in Tags:
          Texts = Tag.text.split("\n")
          for Text in Texts:
            if Text.strip() != "":
              t.append(Text.strip())

      row.append(t)

    return_list.append(row)

    # PackageInsertNoの再取得
    return_list[n][0] = [Filename.split('/')[-1].split(".")[0].split("_",1)[-1]]

    # physchemofactingredientsから一般名が取得可能な場合は取得
    try:
      return_list[n][7] = [return_list[n][7][1]]
    except:
      continue

  return return_list

In [7]:
# XMLファイルから必要なデータを読み込んで保存
Lists_SGML = ReadSGML(sgmls)
df=pd.DataFrame(Lists_SGML)
df.to_csv("ReadSGML.csv")

  0%|          | 0/10152 [00:00<?, ?it/s]

## XML+SGML統合処理

In [8]:
# YJコードをキーとしたリストを再作成
ListsByYJ = []
for L in Lists_XML:
  for YJ in L[2]:
    x = []
    x.append(YJ)
    x += L
    ListsByYJ.append(x)
for L in Lists_SGML:
  for YJ in L[2]:
    x = []
    x.append(YJ)
    x += L
    ListsByYJ.append(x)

In [9]:
# データの確認
print('*** 最初の10件 ***')
for t in ListsByYJ[0:10]:
  print(t)

print('*** 最後の10件 ***')
for t in ListsByYJ[-10:]:
  print(t)

*** 最初の10件 ***
['1190012F3320', ['1190012F3320_1_09'], ['ドネペジル塩酸塩OD錠3mg「テバ」', 'ドネペジル塩酸塩OD錠5mg「テバ」', 'ドネペジル塩酸塩OD錠10mg「テバ」'], ['1190012F3320', '1190012F4327', '1190012F6230'], ['Donepezil Hydrochloride OD Tablets “TEVA”', 'Donepezil Hydrochloride OD Tablets “TEVA”', 'Donepezil Hydrochloride OD Tablets “TEVA”'], ['ドネペジル塩酸塩口腔内崩壊錠'], ['アルツハイマー型認知症及びレビー小体型認知症における認知症症状の進行抑制'], ['〈アルツハイマー型認知症における認知症症状の進行抑制〉', '通常、成人にはドネペジル塩酸塩として1日1回3mgから開始し、1～2週間後に5mgに増量し、経口投与する。高度のアルツハイマー型認知症患者には、5mgで4週間以上経過後、10mgに増量する。なお、症状により適宜減量する。', '〈レビー小体型認知症における認知症症状の進行抑制〉', '通常、成人にはドネペジル塩酸塩として1日1回3mgから開始し、1～2週間後に5mgに増量し、経口投与する。5mgで4週間以上経過後、10mgに増量する。なお、症状により5mgまで減量できる。投与開始12週間後までを目安に、認知機能検査、患者及び家族・介護者から自他覚症状の聴取等による有効性評価を行い、認知機能、精神症状・行動障害、日常生活動作等を総合的に評価してベネフィットがリスクを上回ると判断できない場合は、投与を中止すること。投与開始12週間後までの有効性評価の結果に基づき投与継続を判断した場合であっても、定期的に有効性評価を行い、投与継続の可否を判断すること。'], ['ドネペジル塩酸塩（Donepezil Hydrochloride）']]
['1190012F4327', ['1190012F3320_1_09'], ['ドネペジル塩酸塩OD錠3mg「テバ」', 'ドネペジル塩酸塩OD錠5mg「テバ」', 'ドネペジル塩酸塩OD錠10mg「テバ」'], ['1190012F332

# 薬価基準収載品目リストの読み込み

In [10]:
# 薬価基準収載品目リストの4ファイルを読み込んで1つのDataFrameに結合
xlsxs = glob.glob(PriceLists_Path + '/*.xlsx', recursive=True)

PriceList = pd.DataFrame()
for xlsxFilePath in xlsxs:
  xlsx_table = pd.read_excel(xlsxFilePath,engine="openpyxl")
  PriceList = pd.concat([PriceList,xlsx_table])

PriceList = PriceList.reset_index(drop = True)
PriceList.to_csv("PriceList.csv")
PriceList

,区分,薬価基準収載医薬品コード,成分名,規格,Unnamed: 4,Unnamed: 5,Unnamed: 6,品名,メーカー名,診療報酬において加算等の算定対象となる後発医薬品,先発医薬品,同一剤形・規格の後発医薬品がある先発医薬品,薬価,経過措置による使用期限,備考
0,歯科用薬剤,2710803U1044,リドカイン塩酸塩・アドレナリン酒石酸水素塩,１．８ｍＬ１管,NaN,NaN,NaN,オーラ注歯科用カートリッジ１．８ｍＬ,ジーシー昭和薬品,後発品,NaN,NaN,73.2,NaN,NaN
1,歯科用薬剤,2710803U2032,リドカイン塩酸塩・アドレナリン酒石酸水素塩,１ｍＬ１管,NaN,NaN,NaN,オーラ注歯科用カートリッジ１．０ｍＬ,ジーシー昭和薬品,後発品,NaN,NaN,69.6,NaN,NaN
2,歯科用薬剤,2710806U1021,リドカイン塩酸塩・アドレナリン,１．８ｍＬ１管,NaN,NaN,NaN,歯科用キシロカインカートリッジ,デンツプライシロナ,NaN,NaN,NaN,79.6,NaN,NaN
3,歯科用薬剤,2710806U1030,リドカイン塩酸塩・アドレナリン,１．８ｍＬ１管,NaN,NaN,NaN,キシレステシンＡ注射液（カートリッジ）,スリーエム ジャパン イノベーション,NaN,NaN,NaN,79.6,NaN,NaN
4,歯科用薬剤,2710806U1056,リドカイン塩酸塩・アドレナリン,１．８ｍＬ１管,NaN,NaN,NaN,エピリド配合注歯科用カートリッジ１．８ｍＬ,ニプロ,NaN,NaN,NaN,66.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13290,内用薬,8219002F1024,メサドン塩酸塩,５ｍｇ１錠,NaN,麻,NaN,メサペイン錠５ｍｇ,帝國製薬,NaN,先発品,NaN,184.8,NaN,NaN
13291,内用薬,8219002F2020,メサドン塩酸塩,１０ｍｇ１錠,NaN,麻,NaN,メサペイン錠１０ｍｇ,帝國製薬,NaN,先発品,NaN,351.2,NaN,NaN
13292,内用薬,8219003G1024,タペンタドール塩酸塩,２５ｍｇ１錠,NaN,麻,NaN,タペンタ錠２５ｍｇ,ヤンセンファーマ,NaN,先発品,NaN,110.7,NaN,NaN
13293,内用薬,8219003G2020,タペンタドール塩酸塩,５０ｍｇ１錠,NaN,麻,NaN,タペンタ錠５０ｍｇ,ヤンセンファーマ,NaN,先発品,NaN,210.1,NaN,NaN


# 成分名をキーとした表の作成

## 薬価基準収載品目リストと添付文書データの統合

In [11]:
# 添付文書データから必要な情報をもとに表形式に成型
ListsByYJDF = pd.DataFrame(ListsByYJ)
IndicationDF = pd.DataFrame()

IndicationDF['薬価基準収載医薬品コード'] = ListsByYJDF[0]
IndicationDF["PackageInsertNo"] = ListsByYJDF[1].apply(";\n".join)
IndicationDF["ApprovalBrandName"] = ListsByYJDF[2].apply(";\n".join)
IndicationDF["GenericName"] = ListsByYJDF[5].apply(";\n".join)
IndicationDF["IndicationsOrEfficacy"] = ListsByYJDF[6].apply(";\n".join)
IndicationDF["GeneralName"] = ListsByYJDF[8].apply(";\n".join)
IndicationDF.to_csv("IndicationDF.csv")

IndicationDF = IndicationDF.drop_duplicates()
IndicationDF

,薬価基準収載医薬品コード,PackageInsertNo,ApprovalBrandName,GenericName,IndicationsOrEfficacy,GeneralName
0,1190012F3320,1190012F3320_1_09,ドネペジル塩酸塩OD錠3mg「テバ」;\nドネペジル塩酸塩OD錠5mg「テバ」;\nドネペジ...,ドネペジル塩酸塩口腔内崩壊錠,アルツハイマー型認知症及びレビー小体型認知症における認知症症状の進行抑制,ドネペジル塩酸塩（Donepezil Hydrochloride）
1,1190012F4327,1190012F3320_1_09,ドネペジル塩酸塩OD錠3mg「テバ」;\nドネペジル塩酸塩OD錠5mg「テバ」;\nドネペジ...,ドネペジル塩酸塩口腔内崩壊錠,アルツハイマー型認知症及びレビー小体型認知症における認知症症状の進行抑制,ドネペジル塩酸塩（Donepezil Hydrochloride）
2,1190012F6230,1190012F3320_1_09,ドネペジル塩酸塩OD錠3mg「テバ」;\nドネペジル塩酸塩OD錠5mg「テバ」;\nドネペジ...,ドネペジル塩酸塩口腔内崩壊錠,アルツハイマー型認知症及びレビー小体型認知症における認知症症状の進行抑制,ドネペジル塩酸塩（Donepezil Hydrochloride）
3,5200035C1043,5200035C1043_2_01,クラシエ桂枝加竜骨牡蛎湯エキス細粒,桂枝加竜骨牡蛎湯,体質の虚弱な人で疲れやすく、興奮しやすいものの次の諸症：神経質、不眠症、小児夜泣き、小児夜尿...,
4,3999039M1022,3999039M1022_1_12,オフェブカプセル100mg;\nオフェブカプセル150mg,ニンテダニブエタンスルホン酸塩,特発性肺線維症;\n全身性強皮症に伴う間質性肺疾患;\n進行性線維化を伴う間質性肺疾患,ニンテダニブエタンスルホン酸塩（Nintedanib Ethanesulfonate）（JAN）
...,...,...,...,...,...,...
46986,261170AQ2020,261170AQ2020_1_04,次亜塩０．1％液「ヨシダ」,次亜塩素酸ナトリウム,,
46987,2149114F1234,2149114F1234_1_06,アムバロ配合錠「日新」,バルサルタン／アムロジピンベシル酸塩配合剤,高血圧症,一般名
46988,2649734N1038,2649734N1038_1_06,ジクロフェナクナトリウムクリーム１％「ユートク」,ジクロフェナクナトリウムクリーム,下記疾患並びに症状の鎮痛・消炎;\n変形性関節症，肩関節周囲炎，腱・腱鞘炎，腱周囲炎，上腕骨...,ジクロフェナクナトリウム（Diclofenac Sodium）
46992,5100101X1146,5100101X1146_1_01,ナカジマ シュクシャ,日本薬局方 シュクシャ,漢方処方の調剤に用いる。;\n漢方処方の調剤に用いる。,


In [12]:
# 添付文書データテーブルと薬価基準収載品目リストテーブルをマージ
# YJコードが薬価基準収載医薬品コードではない薬剤はここで落ちる

JoinedDF = pd.merge(PriceList, IndicationDF, on='薬価基準収載医薬品コード', how='inner' )

JoinedDF[['成分名', '区分', 'IndicationsOrEfficacy','薬価基準収載医薬品コード', 'PackageInsertNo','GeneralName']]

,成分名,区分,IndicationsOrEfficacy,薬価基準収載医薬品コード,PackageInsertNo,GeneralName
0,リドカイン塩酸塩・アドレナリン酒石酸水素塩,歯科用薬剤,歯科領域における浸潤麻酔又は伝達麻酔,2710803U1044,2710803U1044_2_05,一般名：
1,リドカイン塩酸塩・アドレナリン酒石酸水素塩,歯科用薬剤,歯科領域における浸潤麻酔又は伝達麻酔,2710803U2032,2710803U1044_2_05,一般名：
2,リドカイン塩酸塩・アドレナリン,歯科用薬剤,歯科領域における浸潤麻酔又は伝達麻酔,2710806U1021,2710806U1021_3_08,リドカイン Lidocaine（JAN）;\nアドレナリン Adrenaline（JAN）
3,リドカイン塩酸塩・アドレナリン,歯科用薬剤,歯科領域における浸潤麻酔又は伝達麻酔,2710806U1030,2710806U1030_2_07,リドカイン Lidocaine（JAN）;\nアドレナリン Adrenaline（JAN）
4,リドカイン塩酸塩・アドレナリン,歯科用薬剤,歯科領域における浸潤麻酔又は伝達麻酔,2710806U1056,2710806U1056_1_03,一般名
...,...,...,...,...,...,...
11963,メサドン塩酸塩,内用薬,他の強オピオイド鎮痛剤で治療困難な下記疾患における鎮痛;\n中等度から高度の疼痛を伴う各種癌,8219002F2020,8219002F1024_1_08,メサドン塩酸塩（JAN）
11964,メサドン塩酸塩,内用薬,他の強オピオイド鎮痛剤で治療困難な下記疾患における鎮痛;\n中等度から高度の疼痛を伴う各種癌,8219002F2020,8219002F1024_2_04,メサドン塩酸塩（JAN）
11965,タペンタドール塩酸塩,内用薬,中等度から高度の疼痛を伴う各種癌における鎮痛,8219003G1024,8219003G1024_1_09,タペンタドール塩酸塩（Tapentadol Hydrochloride）
11966,タペンタドール塩酸塩,内用薬,中等度から高度の疼痛を伴う各種癌における鎮痛,8219003G2020,8219003G1024_1_09,タペンタドール塩酸塩（Tapentadol Hydrochloride）


## 成分名で集約

In [13]:
# 成分名/区分/効能または効果でグルーピングする。YJコードとPackageInsertNoは区切り値で結合集約
# 同じ成分で効能または効果の記載が異なる場合は集約しない（当該添付文書へのリンクを維持するため）
Label_JP = JoinedDF.groupby(['成分名', '区分', 'IndicationsOrEfficacy'])[['薬価基準収載医薬品コード', 'PackageInsertNo']].agg(';\n'.join).reset_index()

with open(datadir+'/Label_JP.pkl', 'wb') as f:
  pickle.dump(Label_JP, f)

Label_JP.to_csv("Label_JP.csv")
Label_JP

,成分名,区分,IndicationsOrEfficacy,薬価基準収載医薬品コード,PackageInsertNo
0,β－ガラクトシダーゼ（アスペルギルス）,内用薬,乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不耐症;\n単一症候性...,2339003B1192,2339003B1192_2_03
1,β－ガラクトシダーゼ（ペニシリウム）,内用薬,乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不耐症単一症候性下痢症...,2339004C1036,2339004C1036_2_05
2,きゅう帰膠艾湯エキス,内用薬,冷え症で、出血過多により、貧血するもの。痔出血、外傷後の内出血、産後出血、貧血症。,5200023C1024,5200023C1024_1_10
3,きゅう帰膠艾湯エキス,内用薬,痔出血,5200023C1032;\n5200023D1020,5200023C1032_2_10;\n5200023D1020_1_09
4,きゅう帰調血飲エキス,内用薬,産後の神経症、体力低下、月経不順,5200024D1024;\n5200024D1024,5200024D1024_1_09;\n5200024D1024_2_07
...,...,...,...,...,...
4399,ｌ－メントール,内用薬,芳香・矯味・矯臭の目的で調剤に用いる。,7149004X1214,7149004X1214_1_02
4400,ｌ－メントール,内用薬,芳香・矯臭・矯味の目的で調剤に用いる。,7149004X1052;\n7149004X1060;\n7149004X1060;\n7...,7149004X1052_1_03;\n7149004X1060_1_03;\n714900...
4401,ｐＨ４処理酸性人免疫グロブリン,注射薬,低又は無ガンマグロブリン血症;\n重症感染症における抗生物質との併用;\n特発性血小板減少性...,6343427A1056;\n6343427A2052;\n6343427A3040;\n6...,6343427A1056_1_06;\n6343427A1056_1_06;\n634342...
4402,ｐＨ４処理酸性人免疫グロブリン,注射薬,無又は低ガンマグロブリン血症;\n慢性炎症性脱髄性多発根神経炎の筋力低下の改善;\n慢性炎症...,6343427A7038;\n6343427A8034;\n6343427A9030;\n6...,6343427H1020_1_02;\n6343427H1020_1_02;\n634342...
